In [1]:
from typing import List, Tuple

# Part 1

In [2]:
def cost(letter: str) -> int:
    if letter == "A":
        return 1
    elif letter == "B":
        return 10
    elif letter == "C":
        return 100
    else:
        return 1000

In [3]:
class Amphipod:
        
    def __init__(self, house):
        self.letter = house.letter
        self.cost = cost(self.letter)

In [4]:
class House:
    
    def __init__(self, letter: str, position: int):
        self.letter = letter
        self.position = position
        self.amphipods = []

    def can_enter(self):
        # print(self.letter, [a.letter for a in self.amphipods])
        return len(self.amphipods) == 0 or all([ (a is None) or (a.letter == self.letter)  for a in self.amphipods])
    
    def is_ok(self) -> bool:
        # print(self.letter, [a.letter for a in self.amphipods], all([a.letter == self.letter for a in self.amphipods]))
        return all([x is not None for x in self.amphipods]) and all([a.letter == self.letter for a in self.amphipods])

In [5]:
def feed_example_houses(house_a, house_b, house_c, house_d):
    house_a.amphipods = [Amphipod(house_b), Amphipod(house_a)]
    house_b.amphipods = [Amphipod(house_c), Amphipod(house_d)]
    house_c.amphipods = [Amphipod(house_b), Amphipod(house_c)]
    house_d.amphipods = [Amphipod(house_d), Amphipod(house_a)]

In [6]:
def feed_real_houses(house_a, house_b, house_c, house_d):
    house_a.amphipods = [Amphipod(house_a), Amphipod(house_b)]
    house_b.amphipods = [Amphipod(house_d), Amphipod(house_c)]
    house_c.amphipods = [Amphipod(house_b), Amphipod(house_d)]
    house_d.amphipods = [Amphipod(house_c), Amphipod(house_a)]

In [7]:
def is_finished(houses: List[House]) -> bool:
    return all([h.is_ok() for h in houses])

In [8]:
def can_move(position: int, target: int, field):
    if target > position:
        for i in range(position + 1, target + 1):
            if isinstance(field[i], Amphipod):
                return False
    else:
        for i in range(target, position):
            if isinstance(field[i], Amphipod):
                return False
    return True

In [9]:
def get_house(houses: List[House], amphi: Amphipod) -> House:
    return [h for h in houses if h.letter == amphi.letter][0]

In [10]:
def clone_element(el):
    if el is None:
        return None
    elif isinstance(el, Amphipod):
        return el
    else: # house
        new_house = House(el.letter, el.position)
        new_house.amphipods = [ a for a in el.amphipods]
        return new_house

In [11]:
def clone_field(field):
    return [ clone_element(el) for el in field]

In [12]:
def unique_value(el) -> str:
    if el is None:
        return ""
    elif isinstance(el, Amphipod):
        return el.letter
    else:
        return "".join([(a.letter if a is not None else "X") for a in el.amphipods])

def serialize(field) -> str:
    """Serialize into a unique deterministic string for the cache."""
    return "[" + "|".join([unique_value(el) for el in field]) + "]"

In [13]:
def insert(amp, house) -> int:
    """
    Insert a value at the futhest None value and reurn the cost for the insertion.
    
    For instance inserting A in [None, None, A, A] becomes [None, A, A, A]
    """
    for i in range(len(house.amphipods) - 1, -1, -1):
        if house.amphipods[i] is None:
            house.amphipods[i] = amp
            return amp.cost * (i + 1)

In [14]:
def extract(house) -> Tuple[Amphipod, int]:
    """
    Extract a value from a house and return the extracted value and the cost for the extraction.
    
    When someone leave the house every body move up except the lowest member of the house.
    For instance in house A: [B, C, A] becomes [C, None, A]
    """
    has_seen_other = False
    previous = None
    cost = 0
    for i in range(len(house.amphipods) - 1, -1, -1):
        amp = house.amphipods[i]
        if (amp is None) or (amp.letter != house.letter):
            has_seen_other = True
        if (has_seen_other):
            house.amphipods[i] = previous
            cost += 0 if amp is None else amp.cost
            previous = amp
    return (previous, cost)

In [15]:
def compute_cost(field, houses, cache = {}) -> cost:
    ser = serialize(field)
    if ser in cache:
        return cache[ser]
    
    # If all house are correct it's over
    if is_finished(houses):
        cache[ser] = 0
        return 0
    
    
    # If anyone can move to their house, do it
    for pos in range(len(field)):
        el = field[pos]
        if isinstance(el, Amphipod):
            house = get_house(houses, el)
            if house.can_enter() and can_move(pos, house.position, field):
                field[pos] = None
                cost = abs(pos - house.position) * el.cost + insert(el, house)
                return cost + compute_cost(field, houses, cache)

    # Otherwise try leaving someone from the house
    min_child = 1000000
    availables = [ i for i in range(len(field)) if field[i] is None]
    for h in houses:
        if not h.can_enter():
            possibles = [i for i in availables if can_move(h.position, i, field)]
            for target in possibles:
                cloned_field = clone_field(field)
                cloned_houses = [cloned_field[house.position] for house in houses]
                house = cloned_field[h.position]
                el, house_cost = extract(house)
                new_cost = abs(h.position - target) * el.cost + house_cost
                cloned_field[target] = el
                child_cost = new_cost + compute_cost(cloned_field, cloned_houses, cache)
                min_child = min(child_cost, min_child)
    cache[ser] = min(cache.get(ser, 10000000), min_child)
    return min_child

In [16]:
def solve_cost(house_feeder) -> int:
    house_a, house_b, house_c, house_d = House("A", 2), House("B", 4), House("C", 6), House("D", 8)
    houses = house_a, house_b, house_c, house_d
    house_feeder(*houses)
    field = [None, None, house_a, None, house_b, None, house_c, None, house_d, None, None]
    return compute_cost(field, houses, {})

In [17]:
assert solve_cost(feed_example_houses) == 12521

In [18]:
solve_cost(feed_real_houses)

13336

# Part 2

In [19]:
def feed_demo_part2(house_a, house_b, house_c, house_d):
    house_a.amphipods = [Amphipod(house_b), Amphipod(house_d), Amphipod(house_d), Amphipod(house_a)]
    house_b.amphipods = [Amphipod(house_c), Amphipod(house_c), Amphipod(house_b), Amphipod(house_d)]
    house_c.amphipods = [Amphipod(house_b), Amphipod(house_b), Amphipod(house_a), Amphipod(house_c)]
    house_d.amphipods = [Amphipod(house_d), Amphipod(house_a), Amphipod(house_c), Amphipod(house_a)]

In [20]:
def feed_real_part2(house_a, house_b, house_c, house_d):
    house_a.amphipods = [Amphipod(house_a), Amphipod(house_d), Amphipod(house_d), Amphipod(house_b)]
    house_b.amphipods = [Amphipod(house_d), Amphipod(house_c), Amphipod(house_b), Amphipod(house_c)]
    house_c.amphipods = [Amphipod(house_b), Amphipod(house_b), Amphipod(house_a), Amphipod(house_d)]
    house_d.amphipods = [Amphipod(house_c), Amphipod(house_a), Amphipod(house_c), Amphipod(house_a)] 

In [21]:
assert solve_cost(feed_demo_part2) == 44169

In [22]:
solve_cost(feed_real_part2)

53308